In [32]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [24]:
class ParentState(TypedDict):

    question: str
    answer_eng: str
    answer_hin: str

In [25]:
parent_llm = ChatOpenAI(model='gpt-4o-mini')
subgraph_llm = ChatOpenAI(model='gpt-4o')

In [26]:
def translate_text(state: ParentState):

    prompt = f"""
        Translate the following text to Hindi.
        Keep it natural and clear. Do not add extra content.

        Text:
        {state["answer_eng"]}
    """.strip()
    
    translated_text = subgraph_llm.invoke(prompt).content

    return {'answer_hin': translated_text}

In [27]:
subgraph_builder = StateGraph(ParentState)

subgraph_builder.add_node('translate_text', translate_text)

subgraph_builder.add_edge(START, 'translate_text')
subgraph_builder.add_edge('translate_text', END)

subgraph = subgraph_builder.compile()

In [28]:
def generate_answer(state: ParentState):

    answer = parent_llm.invoke(f"You are a helpful assistant. Answer clearly.\n\nQuestion: {state['question']}").content
    return {'answer_eng': answer}

In [29]:
parent_builder = StateGraph(ParentState)

parent_builder.add_node("answer", generate_answer)
parent_builder.add_node("translate", subgraph)

parent_builder.add_edge(START, 'answer')
parent_builder.add_edge('answer', 'translate')
parent_builder.add_edge('translate', END)

In [30]:
graph = parent_builder.compile()

In [33]:
final_state = graph.invoke({'question': 'What is quantum physics'})

sections = []
for key, value in final_state.items():
    sections.append(f"### **{key.capitalize()}**\n{value}\n")

output_text = "\n---\n".join(sections)
display(Markdown(output_text))

### **Question**
What is quantum physics

---
### **Answer_eng**
Quantum physics, also known as quantum mechanics, is a fundamental branch of physics that deals with the behavior and interactions of matter and energy at the smallest scales, typically at the level of atoms and subatomic particles. It describes phenomena that classical physics cannot explain, such as:

1. **Wave-Particle Duality**: Particles can exhibit both wave-like and particle-like properties.
2. **Uncertainty Principle**: It is impossible to precisely measure both the position and momentum of a particle simultaneously.
3. **Quantization**: Certain properties, like energy, can only take on discrete values, rather than a continuous range.
4. **Superposition**: Particles can exist in multiple states at once until measured.
5. **Entanglement**: Particles can become correlated in such a way that the state of one (no matter the distance) instantly affects the state of another.

Quantum physics has profound implications, leading to the development of technologies such as semiconductors, lasers, and quantum computing. It challenges our classical intuitions about reality and continues to be an area of active research and philosophical inquiry.

---
### **Answer_hin**
क्वांटम भौतिकी, जिसे क्वांटम यांत्रिकी भी कहा जाता है, भौतिकी की एक मौलिक शाखा है जो पदार्थ और ऊर्जा के सबसे छोटे स्तरों, आमतौर पर परमाणु और उप-परमाणु कणों के स्तर पर व्यवहार और अंतःक्रियाओं का अध्ययन करती है। यह उन घटनाओं का वर्णन करती है जो शास्त्रीय भौतिकी से समझी नहीं जा सकती, जैसे:

1. **तरंग-कण द्वैतता**: कण तरंग-जैसी और कण-जैसी दोनों विशेषताएँ प्रदर्शित कर सकते हैं।
2. **अनिश्चितता सिद्धांत**: यह कण की स्थिति और संवेग, दोनों को एक साथ सटीकता से मापना असंभव है।
3. **मात्राकरण**: कुछ गुणधर्म, जैसे ऊर्जा, केवल निश्चित मान ले सकते हैं, लगातार नहीं।
4. **सुपरपोजिशन**: कण कई स्थितियों में एक साथ मौजूद हो सकते हैं जब तक कि उनका मापन नहीं किया जाता।
5. **संबद्धता**: कण इस तरह से संबंधित हो सकते हैं कि एक के स्थिति (चाहे दूरी कोई भी हो) तुरंत दूसरे की स्थिति को प्रभावित करती है।

क्वांटम भौतिकी के गहरे प्रभाव हैं, जिसने अर्धचालक, लेजर और क्वांटम कंप्यूटिंग जैसी प्रौद्योगिकियों के विकास को प्रेरित किया है। यह हमारी शास्त्रीय वास्तविकता की धारणाओं को चुनौती देती है और सक्रिय अनुसंधान और दार्शनिक अन्वेषण का क्षेत्र बनी हुई है।
